In [ ]:
# Importing the required libraries

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import joblib
import os

In [ ]:
# Creating our X_train array 
import csv

Euler_KAM = []
for file_index in range(1, 20):
    with open(f"IPF\IPF_Final_{file_index}.csv", "r") as file:
        contents = csv.DictReader(file, delimiter=",")
        Euler_at_fixed_strain = []
        for row in contents:
            Euler_at_fixed_strain.append(float(row["Phi2"])) 
        Euler_KAM.append(Euler_at_fixed_strain)
        
X_train = []
for j in range(len(Euler_KAM[1])):
    y_values = np.array([i[j] for i in Euler_KAM]) 
    X_train.append(y_values)

# Reshape for RNN input
# As we always have a 3-D array as an input to a LSTM network
# The first dimension represents batch size
# The second dimension represents time steps
# The third dimension represents the number of units in one input sequence


In [ ]:
np.shape(X_train)

In [ ]:
# Creating our Y _train array
import pandas as pd
Y_train = []
df = pd.read_csv("IPF\IPF_Final_20.csv")
for i in range(len(df['Phi2'])):
    Y_train.append(df['Phi2'].iloc[i])

In [ ]:
np.shape(Y_train)

In [ ]:
# Splitting our data into training-testing batches
# and adjusting their shapes according to the model requirements
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)

In [ ]:
x_train = np.array(x_train)
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
y_train = np.array(y_train)
y_train = y_train.reshape((y_train.shape[0], 1,1))

In [ ]:
x_test = np.array(x_test)
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
y_test = np.array(y_test)
y_test = y_test.reshape((y_test.shape[0], 1,1))

In [ ]:
print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_test))
print(np.shape(y_test))

In [ ]:
# Initialization and training of our model

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Initializing the model here

model = Sequential()
model.add(LSTM(32, input_shape=(19, 1), return_sequences=True))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Print model summary
model.summary()

# Train the model
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test))

# Save the model
model.save("Shear_Phi2_lstm.h5")

In [ ]:
# Defining a function that would predict the data for a given Euler Angle ("Euler") and a given file index ("to_predict")
import pandas as pd
from keras.models import load_model
import csv

def predictions(to_predict, Euler):
    Euler_KAM = []
    for file_index in range(1, 20):
        with open(fr"IPF\IPF_Final_{file_index}.csv", "r") as file:
            contents = csv.DictReader(file, delimiter=",")
            Euler_at_fixed_strain = []
            for row in contents:
                Euler_at_fixed_strain.append(float(row[f"Phi{Euler}"])) 
            Euler_KAM.append(Euler_at_fixed_strain)

    X_train = []
    for j in range(len(Euler_KAM[1])):
        y_values = np.array([i[j] for i in Euler_KAM]) 
        X_train.append(y_values)

    X_copy = X_train.copy()
    outer_len = len(X_copy)
    for file_index in range(20, to_predict):
        df = pd.read_csv(fr"IPF\IPF_{file_index}_pred.csv")
        twenty = df[f"Phi{Euler}"] # Euler can take values 1,_,2

        for i in range(outer_len):
            X_copy[i] = np.delete(X_copy[i], 0)
            X_copy[i] = np.append(X_copy[i], twenty[i])

    path = f"Shear_Phi{Euler}_lstm.h5"
    model = load_model(path)
        
    predicted_values=[]

    for i in range(outer_len):
        pred = model.predict(X_copy[i].reshape(1,19,1))
        predicted_values.append(pred)
        print(i)

        
    predicted_values = np.squeeze(predicted_values, axis=2)
    return predicted_values


In [ ]:
# Taking user input for the file_index for which the predictions are to be made
to_predict = int(input("Enter the strain value for which the predictions are to be made: "))

In [ ]:
# Making the predictions for the required
Phi1 = predictions(to_predict, 1)

In [ ]:
df = pd.DataFrame(Phi1)
df.to_csv(fr"IPF\IPF_{to_predict}_pred.csv")

In [ ]:
Phi = predictions(to_predict, "")

In [ ]:
df = pd.DataFrame(Phi)
df.to_csv(fr"IPF\Phi_{to_predict}_pred.csv")

In [ ]:
Phi2 = predictions(to_predict, 2)

In [ ]:
df = pd.DataFrame(Phi2)
df.to_csv(fr"IPF\Phi2_{to_predict}_pred.csv")